In [ ]:
%pip install captum
%pip install torchcam

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.models as models
from torchvision import datasets, transforms

import time
import os
from IPython.display import display

import PIL.Image as Image
from captum.attr import Occlusion
from captum.attr import visualization as viz

from torchvision.io.image import read_image
from torchvision.transforms.functional import normalize, resize, to_pil_image
from torchcam.methods import GradCAMpp
from torchcam.utils import overlay_mask

In [ ]:
model.eval()

loader = transforms.Compose([transforms.Resize((416, 416)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

random_image_name = 'Aston Martin V8 Vantage Convertible 2012/02412.jpg' #mettre une vraie image random et intégrer l'explicabilité ici
image = Image.open(os.path.join(path_test, random_image_name))

img = loader(image).float()
img = torch.autograd.Variable(img, requires_grad=True)
img = img.unsqueeze(0)
img = img.cuda()

In [ ]:
display(image)

output = model(img)
output = F.softmax(output, dim=1)

prediction_score, pred_label_idx = torch.topk(output, 1)
pred_label_idx.squeeze_()
occlusion = Occlusion(model)

attributions_occ = occlusion.attribute(img,
                                       strides = (3, 8, 8),
                                       target=pred_label_idx,
                                       sliding_window_shapes=(3,15, 15),
                                       baselines=0)

attributions_occ = attributions_occ.squeeze().cpu().detach().numpy()

_ = viz.visualize_image_attr(np.transpose(attributions_occ, (1,2,0)),
                                      method="heat_map",
                                      sign="all",
                                      show_colorbar=True
                                     )

In [ ]:
with GradCAMpp(model) as cam_extractor:
  # Preprocess your data and feed it to the model
  out = model(img)
  # Retrieve the CAM by passing the class index and the model output
  activation_map = cam_extractor(out.squeeze(0).argmax().item(), out)

result = overlay_mask(image, to_pil_image(activation_map[0].squeeze(0), mode='F'), alpha=0.5)
plt.imshow(result); plt.axis('off'); plt.tight_layout(); plt.show()